In [1]:
import pyodbc
import pandas as pd

DB_CONN_STR = (
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=BHARAT_DUNGRIYA\\SQLEXPRESS;"
    "DATABASE=FinGuardDB;"
    "Trusted_Connection=yes;"
    "Encrypt=Optional;"
    "TrustServerCertificate=yes;"
)


In [2]:
def get_last_loaded_date():
    conn = pyodbc.connect(DB_CONN_STR)
    cur = conn.cursor()

    cur.execute("""
        SELECT LastLoadedDate
        FROM etl_load_control
        WHERE TableName = 'fraud_transactions'
    """)

    last_date = cur.fetchone()[0]
    conn.close()
    return last_date


In [3]:
def extract_incremental_data(file_path, last_loaded_date):
    df = pd.read_csv(file_path)

    df["trans_date_trans_time"] = pd.to_datetime(
        df["trans_date_trans_time"],
        errors="coerce"
    )

    # 🔥 FILTER NEW DATA ONLY
    df_new = (
    df[df["trans_date_trans_time"] > last_loaded_date]
    .copy()
    .reset_index()
    .rename(columns={"index": "RowNum"})
)


    return df_new


In [4]:
df_new = extract_incremental_data(
    r"C:\Users\bhara\OneDrive\Desktop\Automated ETL & Data Quality Pipeline for Financial Transactions\fraudTrain.csv",
    get_last_loaded_date()
)

if df_new.empty:
    print("✅ No new data found. ETL skipped.")
    exit()


In [5]:
from Financial_data_anomalies_Check import *

FILE_PATH = r"C:\Users\bhara\OneDrive\Desktop\Automated ETL & Data Quality Pipeline for Financial Transactions\fraudTrain.csv"

last_date = get_last_loaded_date()
df_new = extract_incremental_data(FILE_PATH, last_date)

if df_new.empty:
    print("✅ No new data. ETL skipped.")
else:
    issues = detect_issues(df_new)
    df_new = clean_data(df_new)

    load_raw(df_new)
    log_issues(issues)
    load_clean(df_new)

    update_last_loaded_date(df_new)
    print("✅ Incremental ETL completed successfully")
#**************************----------------------****************************----------------------------**************************************



# for sending mails for issues 
import pyodbc
import pandas as pd

def fetch_latest_issues():
    """
    Fetch the issues logged in the last ETL run.
    """
    conn = pyodbc.connect(DB_CONN_STR)
    query = """
        SELECT RowNum, ColumnName, IssueType, IssueDescription
        FROM dq_issues
        WHERE DetectedAt >= DATEADD(MONTH, -1, GETDATE())  -- last 1 month
        ORDER BY RowNum
    """
    df_issues = pd.read_sql(query, conn)
    conn.close()
    return df_issues
# Step 1: Get latest issues from SQL Server
df_issues = fetch_latest_issues()

# Step 2: Check if there are any issues
if df_issues.empty:
    print("✅ No new data quality issues found.")
else:
    # Step 3: Write to CSV
    output_file = r"C:\Users\bhara\OneDrive\Desktop\Automated ETL & Data Quality Pipeline for Financial Transactions\DQ_ISSUE.csv"
    df_issues.to_csv(output_file, index=False)
    print(f"✅ Latest issues saved to CSV: {output_file}")


✅ Connected to SQL Server successfully!
Total rows to load: 6025


c:\Users\bhara\OneDrive\Desktop\Automated ETL & Data Quality Pipeline for Financial Transactions\Financial_data_anomalies_Check.py:334: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.rename(columns={"index": "RowNum"}, inplace=True)


✅ Incremental ETL completed successfully
✅ Latest issues saved to CSV: C:\Users\bhara\OneDrive\Desktop\Automated ETL & Data Quality Pipeline for Financial Transactions\DQ_ISSUE.csv


C:\Users\bhara\AppData\Local\Temp\ipykernel_6564\1419583833.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_issues = pd.read_sql(query, conn)
